In [1]:
pip install git+https://github.com/pulakeshpradhan/export_earth.git

  Cloning https://github.com/pulakeshpradhan/export_earth.git to c:\users\pulak\appdata\local\temp\pip-req-build-xrpowu9t
  Resolved https://github.com/pulakeshpradhan/export_earth.git to commit e542b8c43a3948ff83b70737be65792141517cf1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for export_earth: filename=export_earth-0.1.0-py3-none-any.whl size=1127 sha256=b000b78ca456a87420a05b0c371f4a126dc6a430e3acc7c6ccc2771671c9d898
  Stored in directory: C:\Users\pulak\AppData\Local\Temp\pip-ephem-wheel-cache-vor6a51c\wheels\07\2b\c5\94b1e8abf447b9a975aad6e97689d72dfdffef195e35922904
Successfully built export_earth
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/pulakeshpradhan/export_earth.git 'C:\Users\pulak\AppData\Local\Temp\pip-req-build-xrpowu9t'


In [1]:
from pulakesh import export_image_to_drive
from pulakesh import download_image_single
from pulakesh import download_image_tiles

In [1]:
import ee
import geemap


# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='pulakeshpradhan')

# Define an image and a region
# image = ee.Image('USGS/SRTMGL1_003')
# region = ee.FeatureCollection("projects/pulakeshpradhan/assets/shp/odisha").geometry()

# Export the image to Google Drive
# export_image_to_drive(image, region, scale=30, description='srtm_export')

# Download the image as a single GeoTIFF
# download_image_single(image, region, scale=30, output_dir='./downloads')

# Download the image as tile
# download_image_tiles(image, region, scale=30, output_dir='./OD', rows=4, cols=4)

In [2]:
import ee
import pandas as pd
import numpy as np


# --------------------------------------
# 2. Define Region of Interest and Time Range
# --------------------------------------
roi = ee.Geometry.Rectangle([91.0, 22.0, 92.0, 23.0])  # Example: Chittagong area
start_date = '2018-01-01'
end_date = '2022-12-31'

# --------------------------------------
# 3. Load and Scale MODIS LST Day Data
# --------------------------------------
modis_lst = ee.ImageCollection('MODIS/006/MOD11A2') \
    .select('LST_Day_1km') \
    .filterDate(start_date, end_date) \
    .map(lambda img: img.multiply(0.02)
         .copyProperties(img, img.propertyNames())
         .set('system:time_start', img.get('system:time_start')))

# --------------------------------------
# 4. Monthly Composite Generator
# --------------------------------------
def get_monthly_composites(collection, reducer='mean'):
    """Generate monthly composites (mean or sum)"""
    def monthly_image(date):
        start = date
        end = date.advance(1, 'month')
        img = collection.filterDate(start, end)
        composite = img.mean() if reducer == 'mean' else img.sum()
        return composite.set('system:time_start', start.millis())

    n_months = ee.Number(ee.Date(end_date).difference(ee.Date(start_date), 'month')).round()
    months = ee.List.sequence(0, n_months)

    return ee.ImageCollection.fromImages(
        months.map(lambda m: monthly_image(ee.Date(start_date).advance(m, 'month')))
    )

# Create monthly composites
monthly_lst = get_monthly_composites(modis_lst, reducer='mean')

c:\Users\pulak\anaconda3\envs\maps\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD11A2! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD11A2

  warnings.warn(warning, category=DeprecationWarning)


## CSV

In [ ]:
# --------------------------------------
# 5. Extract Time Series to DataFrame
# --------------------------------------
def extract_time_series_to_df(collection, band_names, region=None, scale=1000):
    """Convert Earth Engine image collection to pandas DataFrame"""
    if region is None:
        region = roi

    def extract(img):
        stats = img.reduceRegion(ee.Reducer.mean(), region, scale, maxPixels=1e13)
        return ee.Feature(None, stats).set('date', img.get('system:time_start'))

    features = collection.map(extract)
    data = features.getInfo()['features']

    df = pd.DataFrame([
        {'date': pd.to_datetime(f['properties']['date'], unit='ms'),
         **{k: v for k, v in f['properties'].items() if k != 'date'}}
        for f in data
    ])

    for band in band_names:
        if band not in df.columns:
            df[band] = np.nan

    return df
df_lst = extract_time_series_to_df(monthly_lst, ['LST_Day_1km'], region=roi, scale=1000)
df_lst.to_csv('modis_lst_day_monthly.csv', index=False)

## Drive

In [24]:
def export_time_series_to_drive(
    collection, 
    region, 
    scale=1000, 
    export_folder='RGEE', 
    export_filename='lst_day_monthly',
    reducer='mean'
):
    """
    Extracts time series as FeatureCollection and exports to Google Drive as CSV.
    Supports reducer options: 'mean', 'sum', 'min', 'max', 'median'.
    """
    # Select reducer
    if reducer == 'mean':
        ee_reducer = ee.Reducer.mean()
    elif reducer == 'sum':
        ee_reducer = ee.Reducer.sum()
    elif reducer == 'min':
        ee_reducer = ee.Reducer.min()
    elif reducer == 'max':
        ee_reducer = ee.Reducer.max()
    elif reducer == 'median':
        ee_reducer = ee.Reducer.median()
    else:
        raise ValueError(f"Unsupported reducer: {reducer}")

    def extract(img):
        stats = img.reduceRegion(ee_reducer, region, scale, maxPixels=1e13)
        return ee.Feature(None, stats).set('date', img.date().format('YYYY-MM-dd'))
    timeseries_fc = collection.map(extract)
    task = ee.batch.Export.table.toDrive(
        collection=timeseries_fc,
        description=f'{export_filename}_CSV',
        folder=export_folder,
        fileNamePrefix=export_filename,
        fileFormat='CSV'
    )
    task.start()
    print(f"Export task for {export_filename} started.")

# Example usage:
export_time_series_to_drive(monthly_lst, roi, scale=1000, reducer='mean')

Export task for lst_day_monthly started.


In [ ]:
def export_time_series_to_drive(collection, region, scale=1000, export_folder='GEE_exports', export_filename='modis_lst_day_monthly'):
    """
    Extracts time series as FeatureCollection and exports to Google Drive as CSV.
    """
    def extract(img):
        stats = img.reduceRegion(ee.Reducer.mean(), region, scale, maxPixels=1e13)
        return ee.Feature(None, stats).set('date', img.date().format('YYYY-MM-dd'))
    timeseries_fc = collection.map(extract)
    task = ee.batch.Export.table.toDrive(
        collection=timeseries_fc,
        description=f'{export_filename}_CSV',
        folder=export_folder,
        fileNamePrefix=export_filename,
        fileFormat='CSV'
    )
    task.start()
    print(f"Export task for {export_filename} started.")

# Example usage:
export_time_series_to_drive(monthly_lst, roi, scale=1000)

## improved4

In [ ]:
def extract_time_series_to_df(collection, band_names, region=None, scale=1000, reducer='mean'):
    """Convert Earth Engine image collection to pandas DataFrame with reducer option"""
    if region is None:
        region = roi  # Ensure you have defined roi in your script

    # Select reducer
    if reducer == 'mean':
        ee_reducer = ee.Reducer.mean()
    elif reducer == 'sum':
        ee_reducer = ee.Reducer.sum()
    elif reducer == 'min':
        ee_reducer = ee.Reducer.min()
    elif reducer == 'max':
        ee_reducer = ee.Reducer.max()
    elif reducer == 'median':
        ee_reducer = ee.Reducer.median()
    else:
        raise ValueError(f"Unsupported reducer: {reducer}")

    def extract(img):
        stats = img.reduceRegion(ee_reducer, region, scale, maxPixels=1e13)
        return ee.Feature(None, stats).set('date', img.get('system:time_start'))

    features = collection.map(extract)
    data = features.getInfo()['features']

    # Convert the extracted data to a pandas DataFrame
    df = pd.DataFrame([
        {'date': pd.to_datetime(f['properties']['date'], unit='ms'),
         **{k: v for k, v in f['properties'].items() if k != 'date'}}
        for f in data
    ])

    # Add missing bands
    for band in band_names:
        if band not in df.columns:
            df[band] = np.nan

    return df

# --------------------------------------
# 5. Drive Time Series and Export
# --------------------------------------
def extract_time_series_to_drive(collection, band_names, region, scale, export_folder='RGEE', export_filename_prefix='time_series_data'):
    def extract(img):
        stats = img.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region,
            scale=scale,
            maxPixels=1e13
        )
        stats = ee.Dictionary(stats)

        # Ensure all band names are present in the output, fill missing with None
        full_stats = ee.Dictionary.fromLists(
            band_names,
            ee.List(band_names).map(lambda b: stats.get(b, None))
        )

        return ee.Feature(None, full_stats).set('date', img.date().format('YYYY-MM-dd'))

    # Extract time series as a FeatureCollection
    timeseries_fc = collection.map(extract)

    # Export to Google Drive
    task = ee.batch.Export.table.toDrive(
        collection=timeseries_fc,
        description=f'{export_filename_prefix}_CSV',
        folder=export_folder,
        fileNamePrefix=export_filename_prefix,
        fileFormat='CSV'
    )
    
    task.start()
    print(f"Export task for {export_filename_prefix} started.")

#


In [20]:
df_lst = extract_time_series_to_df(monthly_lst, ['LST_Day_1km'], region=roi, scale=1000, reducer='median')
df_lst = extract_time_series_to_df(monthly_lst, ['LST_Day_1km'], region=roi, scale=1000, reducer='mean')
df_lst.to_csv('modis_lst_day_monthly.csv', index=False)
df_lst


#Call the function to extract time series and export to Google Drive
# extract_time_series_to_drive(monthly_lst, ['LST_Day_1km'], roi, scale=1000, export_folder='GEE_exports', export_filename_prefix='modis_lst_day_monthly')



,date,LST_Day_1km
0,2018-01-01,295.676149
1,2018-02-01,300.408025
2,2018-03-01,303.677676
3,2018-04-01,303.311043
4,2018-05-01,300.331967
...,...,...
56,2022-09-01,301.483399
57,2022-10-01,300.887924
58,2022-11-01,301.042101
59,2022-12-01,NaN


In [9]:
df_lst = extract_time_series_to_fc(monthly_lst, ['LST_Day_1km'], region=roi, scale=1000)


TypeError: extract_time_series_to_fc() got multiple values for argument 'region'